# Get 20 packet from each flow

In [55]:
import pandas as pd
import numpy as np
import csv
import io
from scipy.stats import zscore
from csv import writer

Fields_name=['Flow_id','ip.len','ip.id','ip.flags','ip.ttl','ip.proto','ip.checksum','ip.src','ip.dst',
      'udp.srcport','udp.dstport','udp.length','udp.checksum','udp.time_relative','udp.time_delta',
      'tcp.dstport','tcp.srcport','frame.number','frame.time','eth.src','eth.dst',
      'tcp.analysis.ack_rtt','Label']
df = pd.read_csv(r"C:\Users\hind.makarem\Desktop\Server files\Test_folders\res\allpacket.csv", chunksize=100000)

total_len = 0
z=1;
for chunk in df:
    previuos_flow=' ';
    #print (str(z));
    z+=1;
    # Do some preprocessing to reduce the memory size of each chunk
    chunk.columns=Fields_name
    #Delete Unnesseccary Columns
    del chunk['tcp.analysis.ack_rtt']
    del chunk['frame.time']
    del chunk['eth.src']
    del chunk['eth.dst']
    del chunk['ip.id']
    del chunk['ip.src']
    del chunk['ip.dst']
    #Get the hexadecimal ready for convert
    chunk['ip.flags'] = chunk['ip.flags'].fillna('0')
    chunk['ip.checksum'] = chunk['ip.checksum'].fillna('0')
    chunk['udp.checksum'] = chunk['udp.checksum'].fillna('0')
    #Convert the hexadecimal columns into dicimal
    chunk['ip.flags']=chunk['ip.flags'].apply(int, base=16)
    chunk['ip.checksum']=chunk['ip.checksum'].apply(int, base=16)
    chunk['udp.checksum']=chunk['udp.checksum'].apply(int, base=16)
    #Put the tcp and udp port numbers as zero if null
    chunk['tcp.dstport'] = chunk['tcp.dstport'].fillna('0')
    chunk['tcp.srcport'] = chunk['tcp.srcport'].fillna('0')
    chunk['udp.dstport'] = chunk['udp.dstport'].fillna('0')
    chunk['udp.srcport'] = chunk['udp.srcport'].fillna('0')
    #Get rid of char in Label column
    spec_chars = ["!",'"',"#","%","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
    for char in spec_chars:
        previuos_flow=' '
        chunk['Label'] = chunk['Label'].str.replace(char, ' ')
    chunk['Label'] = chunk['Label'].str.replace('B', 'b')
    #Intiate some  variables
    previuos_flow='Nan'
    prev_row=[];
    cur_row=[];
    n=20;
    labels=chunk['Label'].values
    flows=chunk['Flow_id'].values
    del chunk['Label']
    del chunk['Flow_id']
    rows=chunk.values;
   
    for i in range(0, chunk['ip.len'].values.shape[0]-n):
        if (flows[i]!=previuos_flow):
            #print(previuos_flow)
            for j in range(i, i+n-1):
                    if ( flows[i]==flows[j] ):
                        prev_row=rows[j]
                        cur_row = np.concatenate((cur_row, prev_row))
                    else:
                        prev_row=[0] * rows.shape[1]
                        cur_row = np.concatenate((cur_row, prev_row))

            previuos_flow=flows[i]
            cur_row = np.concatenate((cur_row, [labels[i]]))
            with open(r"C:\Users\hind.makarem\Desktop\Server files\Test_folders\n_packet_file\Npacket.csv", 'a', newline='') as f_object2:  
                                # Pass the CSV  file object to the writer() function
                                writer_object = writer(f_object2)
                                # Result - a writer object
                                # Pass the data in the list as an argument into the writerow() function
                                writer_object.writerow(cur_row)
                                cur_row=[]

C:\Users\hind.makarem\AppData\Local\Temp\ipykernel_8420\3819798893.py:55: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  chunk['Label'] = chunk['Label'].str.replace(char, ' ')
C:\Users\hind.makarem\AppData\Local\Temp\ipykernel_8420\3819798893.py:55: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  chunk['Label'] = chunk['Label'].str.replace(char, ' ')


In [56]:
chunk

,ip.len,ip.flags,ip.ttl,ip.proto,ip.checksum,udp.srcport,udp.dstport,udp.length,udp.checksum,udp.time_relative,udp.time_delta,tcp.dstport,tcp.srcport,frame.number
100000,347,64,255,17,31730,1900.0,1900.0,327.0,18426,71424.569330,0.050709,0,0,19502
100001,341,64,255,17,31731,1900.0,1900.0,321.0,36346,71424.619810,0.050479,0,0,19503
100002,341,64,255,17,31726,1900.0,1900.0,321.0,36346,71424.670270,0.050464,0,0,19504
100003,338,64,255,17,28470,1900.0,1900.0,318.0,4815,71476.611370,51.941098,0,0,19513
100004,338,64,255,17,28466,1900.0,1900.0,318.0,4815,71476.662090,0.050727,0,0,19514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142226,76,64,64,17,8140,123.0,123.0,56.0,21963,4562.000265,266.000355,0,0,3856
142227,76,64,64,17,58987,123.0,123.0,56.0,17757,4833.000850,271.000585,0,0,3910
142228,76,64,64,17,40283,123.0,123.0,56.0,21107,5097.001887,264.001037,0,0,3960
142229,76,64,64,17,32463,123.0,123.0,56.0,64934,5638.002110,540.899531,0,0,4072


In [44]:
import pandas as pd
df = pd.read_csv(r"C:\Users\hind.makarem\Desktop\Server files\Test_folders\n_packet_file\Npacket.csv")


C:\Users\hind.makarem\AppData\Local\Temp\ipykernel_19348\1163310803.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\hind.makarem\Desktop\Server files\Test_folders\n_packet_file\Npacket.csv")


In [45]:
df

,60.0,64,64.0,6.0,43149,0,0.1,nan,0.2,nan.1,...,0.202,0.203,0.204,0.205,0.206,0.207,0.208,0.209,0.210,Malicious C&C
0,60.0,64.0,64.0,6.0,43150.0,0.0,0.0,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,Malicious C&C
1,60.0,64.0,64.0,6.0,43150.0,0.0,0.0,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,Malicious C&C
2,60.0,64.0,64.0,6.0,43150.0,0.0,0.0,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,Malicious C&C
3,60.0,64.0,64.0,6.0,8000.0,0.0,0.0,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,Malicious C&C
4,60.0,64.0,64.0,6.0,8000.0,0.0,0.0,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,Malicious C&C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674575,76.0,64.0,64.0,17.0,24379.0,123.0,123.0,56.0,4510.0,0.0,...,123.0,123.0,56.0,13719.0,2241.999687,130.983057,0.0,0.0,3363.0,benign
674576,76.0,64.0,64.0,17.0,61773.0,123.0,123.0,56.0,7909.0,0.0,...,123.0,123.0,56.0,60959.0,2319.999734,129.988631,0.0,0.0,3388.0,benign
674577,76.0,64.0,64.0,17.0,49066.0,123.0,123.0,56.0,15989.0,0.0,...,123.0,123.0,56.0,47151.0,2370.999649,130.998058,0.0,0.0,3396.0,benign
674578,76.0,64.0,64.0,17.0,61885.0,123.0,123.0,56.0,810.0,0.0,...,123.0,123.0,56.0,38002.0,3665.998268,513.000790,0.0,0.0,3855.0,benign


In [46]:
df['    Malicious   C&C'].value_counts()

    benign                                         311386
    Malicious   DDoS                               149734
 empty    benign                                    89373
    Malicious   Okiru                               34061
 empty    Malicious   C&C                           16448
    Malicious   C&C                                 15164
 empty    Malicious   PartOfAHorizontalPortScan     10380
    Malicious   PartOfAHorizontalPortScan            1362
benign                                               1317
    Malicious   C&C Heartbeat                         795
    Malicious   C&C FileDownload                      190
    Malicious   C&C Torii                              48
    Malicious   FileDownload                           12
Name:     Malicious   C&C, dtype: int64

# Delete empty rows and empty columns

In [63]:
df=df.dropna(how = 'all',axis=1)
df=df.dropna(how = 'all')

# Adding fields name to columns

In [118]:
b=[]
for i in range(1,20):
    for j in range(0,14):
        a=chunk.columns[j]+str(i)
        b=b+[a]

In [119]:
b

['ip.len1',
 'ip.flags1',
 'ip.ttl1',
 'ip.proto1',
 'ip.checksum1',
 'udp.srcport1',
 'udp.dstport1',
 'udp.length1',
 'udp.checksum1',
 'udp.time_relative1',
 'udp.time_delta1',
 'tcp.dstport1',
 'tcp.srcport1',
 'frame.number1',
 'ip.len2',
 'ip.flags2',
 'ip.ttl2',
 'ip.proto2',
 'ip.checksum2',
 'udp.srcport2',
 'udp.dstport2',
 'udp.length2',
 'udp.checksum2',
 'udp.time_relative2',
 'udp.time_delta2',
 'tcp.dstport2',
 'tcp.srcport2',
 'frame.number2',
 'ip.len3',
 'ip.flags3',
 'ip.ttl3',
 'ip.proto3',
 'ip.checksum3',
 'udp.srcport3',
 'udp.dstport3',
 'udp.length3',
 'udp.checksum3',
 'udp.time_relative3',
 'udp.time_delta3',
 'tcp.dstport3',
 'tcp.srcport3',
 'frame.number3',
 'ip.len4',
 'ip.flags4',
 'ip.ttl4',
 'ip.proto4',
 'ip.checksum4',
 'udp.srcport4',
 'udp.dstport4',
 'udp.length4',
 'udp.checksum4',
 'udp.time_relative4',
 'udp.time_delta4',
 'tcp.dstport4',
 'tcp.srcport4',
 'frame.number4',
 'ip.len5',
 'ip.flags5',
 'ip.ttl5',
 'ip.proto5',
 'ip.checksum5',
 'u

In [48]:
df.columns=b+['Label']
df.columns

Index(['ip.len1', 'ip.flags1', 'ip.ttl1', 'ip.proto1', 'ip.checksum1',
       'udp.srcport1', 'udp.dstport1', 'udp.length1', 'udp.checksum1',
       'udp.time_relative1',
       ...
       'udp.srcport19', 'udp.dstport19', 'udp.length19', 'udp.checksum19',
       'udp.time_relative19', 'udp.time_delta19', 'tcp.dstport19',
       'tcp.srcport19', 'frame.number19', 'Label'],
      dtype='object', length=267)

In [49]:
df

,ip.len1,ip.flags1,ip.ttl1,ip.proto1,ip.checksum1,udp.srcport1,udp.dstport1,udp.length1,udp.checksum1,udp.time_relative1,...,udp.srcport19,udp.dstport19,udp.length19,udp.checksum19,udp.time_relative19,udp.time_delta19,tcp.dstport19,tcp.srcport19,frame.number19,Label
0,60.0,64.0,64.0,6.0,43150.0,0.0,0.0,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,Malicious C&C
1,60.0,64.0,64.0,6.0,43150.0,0.0,0.0,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,Malicious C&C
2,60.0,64.0,64.0,6.0,43150.0,0.0,0.0,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,Malicious C&C
3,60.0,64.0,64.0,6.0,8000.0,0.0,0.0,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,Malicious C&C
4,60.0,64.0,64.0,6.0,8000.0,0.0,0.0,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,Malicious C&C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674575,76.0,64.0,64.0,17.0,24379.0,123.0,123.0,56.0,4510.0,0.0,...,123.0,123.0,56.0,13719.0,2241.999687,130.983057,0.0,0.0,3363.0,benign
674576,76.0,64.0,64.0,17.0,61773.0,123.0,123.0,56.0,7909.0,0.0,...,123.0,123.0,56.0,60959.0,2319.999734,129.988631,0.0,0.0,3388.0,benign
674577,76.0,64.0,64.0,17.0,49066.0,123.0,123.0,56.0,15989.0,0.0,...,123.0,123.0,56.0,47151.0,2370.999649,130.998058,0.0,0.0,3396.0,benign
674578,76.0,64.0,64.0,17.0,61885.0,123.0,123.0,56.0,810.0,0.0,...,123.0,123.0,56.0,38002.0,3665.998268,513.000790,0.0,0.0,3855.0,benign


# Labels cleaning

In [50]:
df['Label']=df['Label'].replace('benign','    benign    ')
df['Label']=df['Label'].replace(' empty    benign    ','    benign    ')

In [51]:
df['Label']=df['Label'].replace('    Malicious   PartOfAHorizontalPortScan    ','    Malicious   PartOfAHorizontalPortScan')

In [52]:
df['Label']=df['Label'].replace(' empty    Malicious   C&C','    Malicious   C&C')

In [53]:
df['Label']=df['Label'].replace('    Malicious   C&C FileDownload','    Malicious   FileDownload')

In [54]:
df['Label']=df['Label'].replace(' empty    Malicious   PartOfAHorizontalPortScan','    Malicious   PartOfAHorizontalPortScan')

In [55]:
df['Label'].value_counts()

    benign                                   402076
    Malicious   DDoS                         149734
    Malicious   Okiru                         34061
    Malicious   C&C                           31612
    Malicious   PartOfAHorizontalPortScan     11742
    Malicious   C&C Heartbeat                   795
    Malicious   FileDownload                    202
    Malicious   C&C Torii                        48
Name: Label, dtype: int64

In [56]:
df['Label'] = df['Label'].dropna()

In [57]:
df.drop(df.index[df['Label'] == 'nan'], inplace = True)

In [131]:
df

,ip.len1,ip.flags1,ip.ttl1,ip.proto1,ip.checksum1,udp.srcport1,udp.dstport1,udp.length1,udp.checksum1,udp.time_relative1,...,udp.srcport19,udp.dstport19,udp.length19,udp.checksum19,udp.time_relative19,udp.time_delta19,tcp.dstport19,tcp.srcport19,frame.number19,Label
0,60.0,64.0,64.0,6.0,43150.0,0.0,0.0,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,Malicious C&C
1,60.0,64.0,64.0,6.0,43150.0,0.0,0.0,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,Malicious C&C
2,60.0,64.0,64.0,6.0,43150.0,0.0,0.0,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,Malicious C&C
3,60.0,64.0,64.0,6.0,8000.0,0.0,0.0,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,Malicious C&C
4,60.0,64.0,64.0,6.0,8000.0,0.0,0.0,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,Malicious C&C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674575,76.0,64.0,64.0,17.0,24379.0,123.0,123.0,56.0,4510.0,0.0,...,123.0,123.0,56.0,13719.0,2241.999687,130.983057,0.0,0.0,3363.0,benign
674576,76.0,64.0,64.0,17.0,61773.0,123.0,123.0,56.0,7909.0,0.0,...,123.0,123.0,56.0,60959.0,2319.999734,129.988631,0.0,0.0,3388.0,benign
674577,76.0,64.0,64.0,17.0,49066.0,123.0,123.0,56.0,15989.0,0.0,...,123.0,123.0,56.0,47151.0,2370.999649,130.998058,0.0,0.0,3396.0,benign
674578,76.0,64.0,64.0,17.0,61885.0,123.0,123.0,56.0,810.0,0.0,...,123.0,123.0,56.0,38002.0,3665.998268,513.000790,0.0,0.0,3855.0,benign


# Replace Nan values with median of each columns

In [64]:
df=df.fillna(df.median())

C:\Users\hind.makarem\AppData\Local\Temp\ipykernel_19348\308181716.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df=df.fillna(df.median())


In [65]:
for i in df.columns:
    a=df[i].isnull().values.any()
    if(a==True):
        print(str(i))
        print(str(a))

In [59]:
df['Label'].isnull().values.any()

False

# Apply Zscore function to all columns except Label column

In [26]:
from scipy.stats import zscore

In [68]:
x_columns = df.columns.drop('Label')
for col in x_columns:
    df[col] = zscore(df[col])

# Display DataFrame after data cleaning

In [69]:
df

,ip.flags1,ip.ttl1,ip.proto1,ip.checksum1,udp.srcport1,udp.dstport1,udp.length1,udp.checksum1,udp.time_relative1,udp.time_delta1,...,udp.srcport19,udp.dstport19,udp.length19,udp.checksum19,udp.time_relative19,udp.time_delta19,tcp.dstport19,tcp.srcport19,frame.number19,Label
0,0.535101,-1.196927,-1.495122,0.465960,-0.392753,-0.342083,0.071326,-0.976813,-0.218557,-0.106434,...,-0.312127,-0.574869,-0.470381,-0.573924,-0.436565,-0.100667,-0.047923,-0.444593,-0.376701,Malicious C&C
1,0.535101,-1.196927,-1.495122,0.465960,-0.392753,-0.342083,0.071326,-0.976813,-0.218557,-0.106434,...,-0.312127,-0.574869,-0.470381,-0.573924,-0.436565,-0.100667,-0.047923,-0.444593,-0.376701,Malicious C&C
2,0.535101,-1.196927,-1.495122,0.465960,-0.392753,-0.342083,0.071326,-0.976813,-0.218557,-0.106434,...,-0.312127,-0.574869,-0.470381,-0.573924,-0.436565,-0.100667,-0.047923,-0.444593,-0.376701,Malicious C&C
3,0.535101,-1.196927,-1.495122,-1.499103,-0.392753,-0.342083,0.071326,-0.976813,-0.218557,-0.106434,...,-0.312127,-0.574869,-0.470381,-0.573924,-0.436565,-0.100667,-0.047923,-0.444593,-0.376701,Malicious C&C
4,0.535101,-1.196927,-1.495122,-1.499103,-0.392753,-0.342083,0.071326,-0.976813,-0.218557,-0.106434,...,-0.312127,-0.574869,-0.470381,-0.573924,-0.436565,-0.100667,-0.047923,-0.444593,-0.376701,Malicious C&C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674575,0.535101,-1.196927,0.668842,-0.583434,-0.383416,-0.300690,-0.996065,-0.722394,-0.814921,-0.106618,...,-0.301177,-0.421887,-0.270952,0.360302,-0.343983,1.071058,-0.047923,-0.444593,-0.355797,benign
674576,0.535101,-1.196927,0.668842,1.507080,-0.383416,-0.300690,-0.996065,-0.530649,-0.814921,-0.106618,...,-0.301177,-0.421887,-0.270952,3.577216,-0.340762,1.062162,-0.047923,-0.444593,-0.355642,benign
674577,0.535101,-1.196927,0.668842,0.796694,-0.383416,-0.300690,-0.996065,-0.074838,-0.814921,-0.106618,...,-0.301177,-0.421887,-0.270952,2.636929,-0.338656,1.071192,-0.047923,-0.444593,-0.355592,benign
674578,0.535101,-1.196927,0.668842,1.513341,-0.383416,-0.300690,-0.996065,-0.931119,-0.814921,-0.106618,...,-0.301177,-0.421887,-0.270952,2.013908,-0.285180,4.488442,-0.047923,-0.444593,-0.352739,benign


# Save DataFrame

In [43]:
df.to_csv(r"C:\Users\hind.makarem\Desktop\Server files\Test_folders\n_packet_file\Npacket_cleaned.csv")